In [ ]:
%load_ext autoreload
%autoreload 2
from scipy.interpolate import BSpline, splev
from multiple_flux_surfaces_class import multiple_flux_surfaces
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from grad_shafranov_class import GS_Solution
import numpy as np
from curve_fitting.curve_fitting_class import curve_fitter
from matplotlib.backends.backend_pdf import PdfFile,PdfPages
from pathlib import Path

In [ ]:
Chosen_form_factor = "Dshape"
# Chosen_form_factor = "spherical tokamak NSTX"
# Chosen_form_factor = "ITER from Cerfon"
# Chosen_form_factor = "circular shaped tokamak"
Form_factor_label = Chosen_form_factor.replace(' ','_')

flux_function = GS_Solution(formfactor=Chosen_form_factor)
curve_fitting_object =curve_fitter(solverClassObject=flux_function)
flux_surfaces = multiple_flux_surfaces(curve_fitting_class_object=curve_fitting_object)

In [ ]:
#getting coefficents of all necessary curves in log space near the axis

m_max_project_beg = 1
m_max_project_end = 15
number_of_s_points = 15

coefficients1 = []
coefficients2 = []
coefficients4 = []
coefficients60 = []
coefficients_no_root_search = []

s = np.logspace(-4,0,number_of_s_points)
for m_final in range(m_max_project_beg, m_max_project_end+1):
    coeffs_for_1m_final = []
    coeffs_for_2m_final = []
    coeffs_for_4m_final = []
    coeffs_for_60m_final = []
    coefficients_no_root_search_final = []
    for i in range(len(s)):
        psi_fs,lcfs_x,xmap=curve_fitting_object.find_flux_surface(s_fs=s[i],m_max_start=m_max_project_beg, m_max_end=m_final)
        
        coefficients_no_root_search_final.append((lcfs_x,xmap))
        
        rr1,zz1 = curve_fitting_object.get_curve(s_fs=s[i],x_final_in=lcfs_x,xmap_in=xmap,Np_in=2*(2*(m_final)+1)+1, rzcorr_in=1)
        # print(f'N_in: {2*(2*m_final+1)+1}')
        rr2,zz2 = curve_fitting_object.get_curve(s_fs=s[i],x_final_in=lcfs_x,xmap_in=xmap,Np_in=2*(2*(2*m_final)+1)+1,rzcorr_in=1)
        # print(f'N_in: {2*(2*(2*m_final)+1)+1}')
        rr4,zz4 = curve_fitting_object.get_curve(s_fs=s[i],x_final_in=lcfs_x,xmap_in=xmap,Np_in=2*(2*(4*m_final)+1)+1,rzcorr_in=1)
        # print(f'N_in: {2*(2*(4*m_final)+1)+1}')
        rr60,zz60 = curve_fitting_object.get_curve(s_fs=s[i],x_final_in=lcfs_x,xmap_in=xmap,Np_in=2*(2*(60)+1)+1,rzcorr_in=1)
        # print(f'N_in: {2*(2*(60)+1)+1}')
        
        coeffs_for_1m_final.append(curve_fitting_object.inverse_fourier_tranform(m_max_end=m_final, rr=rr1, zz=zz1))
        coeffs_for_2m_final.append(curve_fitting_object.inverse_fourier_tranform(m_max_end=2*m_final, rr=rr2, zz=zz2))
        coeffs_for_4m_final.append(curve_fitting_object.inverse_fourier_tranform(m_max_end=4*m_final, rr=rr4, zz=zz4))
        coeffs_for_60m_final.append(curve_fitting_object.inverse_fourier_tranform(m_max_end=60, rr=rr60, zz=zz60))
        
    
    coefficients1.append(coeffs_for_1m_final)
    coefficients2.append(coeffs_for_2m_final)
    coefficients4.append(coeffs_for_4m_final)
    coefficients60.append(coeffs_for_60m_final)
    coefficients_no_root_search.append(coefficients_no_root_search_final)
    
    print(f'm_final:{m_final}')

In [ ]:
#plotting error curves without root search 
fig, ax = plt.subplots()


colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))
for m_final in range(m_max_project_beg, m_max_project_end+1):
    error_curve_no_root_search = np.zeros(number_of_s_points)
    for i in range(number_of_s_points):
        x,xmap =  coefficients_no_root_search[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve_no_root_search[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
        
    ax.plot(s,error_curve_no_root_search, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('s', fontsize=15)
ax.set_ylabel('relative error in s', fontsize=15)
plt.legend(bbox_to_anchor = (1,1))



path = Path().parent / f"../plots_pdfs/{Form_factor_label}_error_curve_without_root_search.pdf"
pdfFile = PdfPages(path)
pdfFile.savefig(fig, bbox_inches='tight')
pdfFile.close()



In [ ]:
#plotting error curves with root search 

fig, ax = plt.subplots(2,2, figsize=(10,10))
colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))

for m_final in range(m_max_project_beg, m_max_project_end+1):
    error_curve1 = np.zeros(number_of_s_points)
    error_curve2 = np.zeros(number_of_s_points)
    error_curve4 = np.zeros(number_of_s_points)
    error_curve60 = np.zeros(number_of_s_points)
    for i in range(number_of_s_points):
        x,xmap =  coefficients1[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve1[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
        
        x,xmap =  coefficients2[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve2[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
        
        x,xmap =  coefficients4[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve4[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
        
        x,xmap =  coefficients60[m_final-m_max_project_beg][i]
        R,Z = curve_fitting_object.eval_curve(x=x, xmap=xmap)
        error_curve60[i] = np.max(np.abs(curve_fitting_object.flux_function.eval_s(R=R, Z=Z)-s[i]))/s[i]
    
    
    ax[0,0].plot(s,error_curve1, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])
    ax[0,1].plot(s,error_curve2, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])
    ax[1,0].plot(s,error_curve4, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])
    ax[1,1].plot(s,error_curve60, label=f'm_final:{m_final}', color = colors[m_final-m_max_project_beg])

f = lambda x: (x.set_xscale('log'), x.set_yscale('log'), x.set_xlabel('s', fontsize=12), x.set_ylabel('Relative error in s',fontsize=12))
np.vectorize(f)(ax)
ax[0,0].legend(mode='expand',ncol = 5, bbox_to_anchor = (0.1,0.3,2,1))

ax[0,0].set_title('(a)')
ax[0,1].set_title('(b)')
ax[1,0].set_title('(c)')
ax[1,1].set_title('(d)')
fig.subplots_adjust(wspace=0.25)

path = Path().parent / f"../plots_pdfs/{Form_factor_label}_error_curve_with_root_search.pdf"
pdfFile = PdfPages(path)
pdfFile.savefig(fig)
pdfFile.close()



In [ ]:
#calculating coefficients at the boundary for m_proj=60
m_max_project_beg = 1
m_max_project_end = 15
number_of_s_points = 15

coeffs_for_s_equal_1 = []
for m_final in range(m_max_project_beg, m_max_project_end+1):
    coeffs_for_s_equal_1.append(curve_fitting_object.get_full_contour(s_fs=1, m_max_start_first=1, m_max_end_first=m_final, m_max_end=60, root_search=True))




In [ ]:

Rhat_0 = np.zeros(len(coeffs_for_s_equal_1))
Zhat_0 = np.zeros(len(coeffs_for_s_equal_1))
# print(coeffs_for_s_equal_1[0][1])

fig, ax = plt.subplots(1,2,figsize=(16,8))
fig2, ax2 = plt.subplots(1,2,figsize=(16,8))

end =20
colors = plt.cm.jet(np.linspace(0,1,end, endpoint=True))

for j in range(end):
    for i in range(len(coeffs_for_s_equal_1)):
        Coeff, coeffs_map = coeffs_for_s_equal_1[i]
        Coeff = np.asarray(Coeff)
        Rhat_0[i] = Coeff[coeffs_map['str_r_c']+j]
        Zhat_0[i] = Coeff[coeffs_map['str_z_c']+j]
        # Rhat_0 = Rhat_0 / Rhat_0[-1]
        # Zhat_0 = Zhat_0 / Zhat_0[-1]
        Rhat_0 = Rhat_0 
        Zhat_0 = Zhat_0 
        
        
    
      
    ax[0].plot(np.linspace(1,15,15, endpoint=True),abs(Rhat_0), color=colors[j], label = f'Rhat_{j}')
    ax2[0].plot(np.linspace(1,15,15, endpoint=True),abs(Zhat_0), color=colors[j], label = f'Zhat_{j}')
    ax[1].plot(np.linspace(1,15,15, endpoint=True),abs((Rhat_0-Rhat_0[-1])/Rhat_0[-1])+1e-16, color=colors[j], label = f'Rhat_{j}')
    ax2[1].plot(np.linspace(1,15,15, endpoint=True),abs((Zhat_0-Zhat_0[-1])/Zhat_0[-1])+1e-16, color=colors[j], label = f'Zhat_{j}')


f = lambda x,y: (x.set_yscale('log'), x.set_xlabel(y, fontsize=13))
np.vectorize(f)(ax,r'$m_{max}$')
np.vectorize(f)(ax2,r'$m_{max}$')
ax[1].legend(bbox_to_anchor=(1.23,1))
ax2[1].legend(bbox_to_anchor=(1.23,1))

ax[0].set_ylabel(r"$\hat{R}$" + ' coefficient values', fontsize=13)
ax2[0].set_ylabel(r"$\hat{Z}$" + ' coefficient values', fontsize=13)

ax[1].set_ylabel(r"$\left|\frac{\hat{R}_{m_{max}}-\hat{R}_{m_{max}}^{ref}}{\hat{R}_{m_{max}}^{ref}}\right| + 10^{-16}$", fontsize=15)
ax2[1].set_ylabel(r"$\left|\frac{\hat{Z}_{m_{max}}-\hat{Z}_{m_{max}}^{ref}}{\hat{Z}_{m_{max}}^{ref}}\right| + 10^{-16}$", fontsize=15)
ax[0].set_title('(a)')
ax[1].set_title('(b)')
ax2[0].set_title('(a)')
ax2[1].set_title('(b)')

path1 = Path().parent / f"../plots_pdfs/{Form_factor_label}_R_coefficient_conrvergence.pdf"
pdfFile1 = PdfPages(path1)
pdfFile1.savefig(fig)
pdfFile1.close()

path2 = Path().parent / f"../plots_pdfs/{Form_factor_label}_Z_coefficient_conrvergence.pdf"
pdfFile2 = PdfPages(path2)
pdfFile2.savefig(fig2)
pdfFile2.close()



In [ ]:
m_max_project_beg = 1
m_max_project_end = 15
number_of_s_points = 15

slog = np.logspace(-4,-1,number_of_s_points, endpoint=True)
slin = np.linspace(0,1,number_of_s_points, endpoint=True)
slin[0]=1e-4

q_data_points_near_axis = np.zeros((m_max_project_end+1-m_max_project_beg,number_of_s_points))
q_data_points_full = np.zeros((m_max_project_end+1-m_max_project_beg,number_of_s_points))

colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))
for m_final in range(m_max_project_beg, m_max_project_end+1):
    print(f'm = {m_final}')
    for i in range(number_of_s_points):
        q_data_points_near_axis[m_final - m_max_project_beg][i] = flux_surfaces.safety_factor_with_dedicated_contour(s=slog[i],m_max_end=60, m_max_start_first=1, m_max_end_first=m_final)
        q_data_points_full[m_final - m_max_project_beg][i] = flux_surfaces.safety_factor_with_dedicated_contour(s=slin[i],m_max_end=60, m_max_start_first=1, m_max_end_first=m_final)
    

In [ ]:
fig, ax = plt.subplots(1,2 ,figsize=(16,8))
colors = plt.cm.jet(np.linspace(0,1,m_max_project_end+1-m_max_project_beg, endpoint=True))
for m_final in range(m_max_project_beg, m_max_project_end):
    ax[0].plot(slog, q_data_points_near_axis[m_final], color = colors[m_final-m_max_project_beg], label=f'm_final:{m_final}')
    ax[1].plot(slin, q_data_points_full[m_final], color = colors[m_final-m_max_project_beg],label=f'm_final:{m_final}')
    ax[0].set_xscale('log')
    ax[0].set_xlabel('s', fontsize=15)
    ax[1].set_xlabel('s', fontsize=15)
    ax[0].set_ylabel('q(Safety Factor)', fontsize=15)
    ax[1].set_ylabel('q(Safety Factor)', fontsize=15)
    ax[1].legend(bbox_to_anchor=(1,1))
    ax[0].set_title('(a)')
    ax[1].set_title('(b)')
    

path1 = Path().parent / f"../plots_pdfs/{Form_factor_label}_q_profile_plot.pdf"
pdfFile1 = PdfPages(path1)
pdfFile1.savefig(fig)
pdfFile1.close()


In [ ]:
colors = plt.cm.jet(np.linspace(0,1,number_of_s_points, endpoint=True))
fig, ax = plt.subplots(1,2,figsize = (18,8))
plot_y_near_axis = np.transpose(abs(q_data_points_near_axis/q_data_points_near_axis[-1]-1)+1.0e-16)
plot_y_full = np.transpose(abs(q_data_points_full/q_data_points_full[-1]-1)+1.0e-16)

for i in range(number_of_s_points):
    ax[0].plot(np.arange(start=m_max_project_beg, stop=m_max_project_end+1),plot_y_near_axis[i],label =f"s:{round(slog[i], 4)}", color=colors[i])
    ax[1].plot(np.arange(start=m_max_project_beg, stop=m_max_project_end+1),plot_y_full[i],label =f"s:{round(slin[i], 4)}", color=colors[i])
    
ax[0].set_xlabel(r'$m_{max}$', fontsize=15)
ax[0].set_ylabel(r'$|\frac{q(s)_{i}}{q(s)_{m_{max}}}-1| + 10^{-16}$',fontsize=15)
ax[1].set_xlabel(r'$m_{max}$', fontsize=15)
ax[1].set_ylabel(r'$|\frac{q(s)_{i}}{q(s)_{m_{max}}}-1| + 10^{-16}$',fontsize=15)
ax[0].legend(mode='expand',ncol = 5, bbox_to_anchor = (0,-1.08,1,1))
ax[1].legend(mode='expand',ncol = 5, bbox_to_anchor = (0,-1.08,1,1))
ax[0].set_yscale('log')
ax[1].set_yscale('log')
ax[0].set_title('(a)')
ax[1].set_title('(b)')

path1 = Path().parent / f"../plots_pdfs/{Form_factor_label}_q_profile_plot_relative_convergence.pdf"
pdfFile1 = PdfPages(path1)
pdfFile1.savefig(fig, bbox_inches='tight')
pdfFile1.close()
